## 実験
2値分類問題として解けるかどうか検証する

In [ ]:
!nvidia-smi

Sun Jan  2 05:18:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/kaggle/petfinder

/content/drive/MyDrive/kaggle/petfinder


In [ ]:
!pip install pytorch-lightning==1.4.0 grad-cam ttach
!pip install hydra-core --upgrade
!pip install wandb

In [ ]:
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
import sys
sys.path.append('/content/drive/MyDrive/kaggle/petfinder/pytorch-image-models')
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image

from omegaconf import OmegaConf

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule

from pytorch_lightning.loggers import WandbLogger
import wandb


warnings.filterwarnings("ignore")

In [ ]:
# wandb setting
wandb.init(
    project="petfinder",
    entity=ENTITY,
    id=ID,
)
wandb_logger = WandbLogger()

wandb: Currently logged in as: takeru (use `wandb login --relogin` to force relogin)


## config

In [ ]:
config = {'seed': 15,
          'root': '/content/drive/MyDrive/kaggle/petfinder/petfinder-pawpularity-score', 
          'n_splits': 5,
          'epoch': 10,
          'trainer': {
              'gpus': 1,
              'accumulate_grad_batches': 2, #要検討
              'progress_bar_refresh_rate': 1,
              'fast_dev_run': False,
              'num_sanity_val_steps': 0,
              'resume_from_checkpoint': None,
          },
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'train_loader':{
              'batch_size': 30,
              'shuffle': True,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': True,
          },
          'val_loader': {
              'batch_size': 30,
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': False
         },
          'model':{
              'name': 'swin_large_patch4_window7_224',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 10,
                  'eta_min': 1e-4,
              }
          }
}

config = OmegaConf.create(config)

## dataset

In [ ]:
class PetfinderDataset(Dataset):
    def __init__(self, df, image_size=224):
        self._X = df["Id"].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            if label >= 38:
              binary_label = 1
            else:
              binary_label = 0
            return image, label, binary_label
        return image

class PetfinderDataModule(LightningDataModule):
    def __init__(
        self,
        train_df,
        val_df,
        cfg,
    ):
        super().__init__()
        self._train_df = train_df
        self._val_df = val_df
        self._cfg = cfg

    def __create_dataset(self, train=True):
        return (
            PetfinderDataset(self._train_df, self._cfg.transform.image_size)
            if train
            else PetfinderDataset(self._val_df, self._cfg.transform.image_size)
        )

    def train_dataloader(self):
        dataset = self.__create_dataset(True)
        return DataLoader(dataset, **self._cfg.train_loader)

    def val_dataloader(self):
        dataset = self.__create_dataset(False)
        return DataLoader(dataset, **self._cfg.val_loader)

    def test_dataloader(self):
        dataset = self.__create_dataset(False)
        return DataLoader(dataset, **self._cfg.val_loader)  

## visualize data

In [ ]:
torch.autograd.set_detect_anomaly(True)
seed_everything(config.seed)

dataset_dir = "/content/drive/MyDrive/kaggle/petfinder/petfinder-pawpularity-score/train"

df = pd.read_csv(os.path.join(config.root, "train.csv"))
df["Id"] = df["Id"].apply(lambda x: os.path.join(dataset_dir, x + ".jpg"))

Global seed set to 15


## augmentation

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB


def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
    }
    return transform


## model

In [ ]:
from sklearn.metrics import accuracy_score

def mixup(x: torch.Tensor, y: torch.Tensor, alpha: float = 1.0):
    assert alpha > 0, "alpha should be larger than 0"
    assert x.size(0) > 1, "Mixup cannot be applied to a single instance."

    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x.size()[0])
    mixed_x = lam * x + (1 - lam) * x[rand_index, :]
    target_a, target_b = y, y[rand_index]
    return mixed_x, target_a, target_b, lam

class Model(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.fold = cfg.fold
        self.__build_model()
        # self._criterion = nn.CrossEntropyLoss(label_smoothing=0.3)
        self._criterion = nn.BCEWithLogitsLoss()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=True, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, self.cfg.model.output_dim)
        )

    def forward(self, x):
        f = self.backbone(x)
        out = self.fc(f)
        return out

    def training_step(self, batch, batch_idx):
        loss, logits, labels, binary_labels = self.__share_step(batch, 'train')
        return {'loss': loss, 'logits': logits, 'labels': labels, 'binary_labels': binary_labels}
        
    def validation_step(self, batch, batch_idx):
        loss, logits, labels, binary_labels = self.__share_step(batch, 'val')
        return {'loss': loss, 'logits': logits, 'labels': labels, 'binary_labels': binary_labels}

    def test_step(self, batch, batch_idx):
        loss, logits, labels, binary_labels = self.__share_step(batch, 'val')
        return {'loss': loss, 'logits': logits, 'labels': labels, 'binary_labels': binary_labels}    
    
    def __share_step(self, batch, mode):
        images, labels, binary_labels = batch
        labels = labels.long()
        binary_labels = binary_labels.float()
        images = self.transform[mode](images)
        
        logits = self.forward(images).squeeze(1)
        loss = self._criterion(logits, binary_labels)

        logits = logits.detach().cpu()
        labels = labels.detach().cpu()
        binary_labels = binary_labels.detach().cpu()        
        return loss, logits, labels, binary_labels
        
    def training_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'train')

    def validation_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'val') 

    def test_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'test')           
        
    def __share_epoch_end(self, outputs, mode):
        preds = []
        labels = []
        binary_labels = []
        binary_pred = []
        for index, out in enumerate(outputs):
            logits, label, binary_label, loss = out['logits'], out['labels'], out["binary_labels"], out["loss"]
            if index == 0:
              loss = loss.detach().cpu().numpy()
              total_loss = loss
            else:
              total_loss += loss.cpu().numpy()
            binary_pred.extend(list(np.where(logits.clone().detach().cpu().numpy() < 0.38, 0, 1)))
            preds.append(logits.clone().sigmoid() * 100.)
            labels.append(label)
            binary_labels.append(binary_label)
        total_loss = total_loss / len(outputs)
        preds = torch.cat(preds).float()
        binary_labels = torch.cat(binary_labels).long().numpy()
        if mode == "test":
          self.oof_preds = preds
        else:
          labels = torch.cat(labels).float()
          metrics = torch.sqrt(((labels - preds) ** 2).mean())
          self.log(f"fold{self.fold}/{mode}_rmse_loss", metrics)

          self.log(f"fold{self.fold}/{mode}_binary_loss", total_loss)          

          accuracy = accuracy_score(binary_pred, binary_labels)
          self.log(f"fold{self.fold}/{mode}_binary_accuracy", accuracy)

    
    def check_gradcam(self, dataloader, target_layer, target_category, reshape_transform=None):
        cam = GradCAMPlusPlus(
            model=self,
            target_layer=target_layer, 
            use_cuda=self.cfg.trainer.gpus, 
            reshape_transform=reshape_transform)
        
        org_images, labels = iter(dataloader).next()
        cam.batch_size = len(org_images)
        images = self.transform['val'](org_images)
        images = images.to(self.device)
        logits = self.forward(images).squeeze(1)
        pred = logits.sigmoid().detach().cpu().numpy()
        labels = labels.cpu().numpy()
        
        grayscale_cam = cam(input_tensor=images, target_category=target_category, eigen_smooth=True)
        org_images = org_images.detach().cpu().numpy().transpose(0, 2, 3, 1) / 255.
        return org_images, grayscale_cam, pred, labels

    def configure_optimizers(self):
        optimizer = eval(self.cfg.optimizer.name)(
            self.parameters(), **self.cfg.optimizer.params
        )
        scheduler = eval(self.cfg.scheduler.name)(
            optimizer,
            **self.cfg.scheduler.params
        )
        return [optimizer], [scheduler]

## train

In [ ]:

skf = StratifiedKFold(
    n_splits=config.n_splits, shuffle=True, random_state=config.seed
)

for fold, (train_idx, val_idx) in enumerate(skf.split(df["Id"], df["Pawpularity"])):
    config.fold = fold
    train_df = df.loc[train_idx].reset_index(drop=True)
    val_df = df.loc[val_idx]
    val_index = val_df.index
    val_df = val_df.reset_index(drop=True)
    datamodule = PetfinderDataModule(train_df, val_df, config)
    model = Model(config)
    earystopping = EarlyStopping(monitor=f"fold{fold}/val_loss")
    lr_monitor = callbacks.LearningRateMonitor()
    loss_checkpoint = callbacks.ModelCheckpoint(
        filename="best_loss",
        monitor=f"fold{fold}/val_binary_loss",
        save_top_k=1,
        mode="min",
        save_last=False,
    )
    logger = wandb_logger
    
    trainer = pl.Trainer(
        logger=logger,
        max_epochs=config.epoch,
        callbacks=[lr_monitor, loss_checkpoint],
        **config.trainer,
    )
    trainer.fit(model, datamodule=datamodule)
    trainer.test(ckpt_path="best")

    if fold == 0:
      oof_predictions = np.zeros(len(df))
    oof_predictions[val_index] = model.oof_preds

    del datamodule, model, trainer

df['oof_pred'] = oof_predictions
df[['Id', 'oof_pred']].to_csv(f'oof_{config.model.name}_bce.csv', index=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | backbone   | SwinTransformer   | 194 M 
1 | fc         | Sequential        | 1.5 K 
2 | _criterion | BCEWithLogitsLoss | 0     
-------------------------------------------------
194 M     Trainable params
0         Non-trainable params
194 M     Total params
779.988   Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | backbone   | SwinTransformer   | 194 M 
1 | fc         | Sequential        | 1.5 K 
2 | _criterion | BCEWithLogitsLoss | 0     
-------------------------------------------------
194 M     Trainable params
0         Non-trainable params
194 M     Total params
779.988   Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]